# RST contrast with continuous Deformable Mirror (DM)
### Goal

After manage our basic contrast tool, we will try to understand how use our DM and measure his influence on contrast.


## Use DM

In [ ]:
#WebbPSF requierement
%pylab inline --no-import-all
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'

import os
os.environ['WEBBPSF_PATH'] = '/home/marken/GitHub/webbpsf_data'
import webbpsf
from webbpsf import roman
import ipywidgets
from astropy.io import fits
import astropy.units as u
import pastis.util as util
from pastis.e2e_simulators.telescopes import RST

In [ ]:
import logging

#Get WebbPSF logs
webbpsf.setup_logging()

#Remove useless logs warning
log = logging.getLogger()
mplfm_logger = logging.getLogger('matplotlib.font_manager')
mplcb_logger = logging.getLogger('matplotlib.colorbar')
mplt_logger = logging.getLogger('matplotlib.ticker')
mplbe_logger = logging.getLogger('matplotlib.backends')

mplfm_logger.setLevel(logging.WARNING)
mplcb_logger.setLevel(logging.WARNING)
mplt_logger.setLevel(logging.WARNING)
mplbe_logger.setLevel(logging.WARNING)

In [ ]:
#Create a coronagraph instancy
SPC = roman.CGI(mode='CHARSPC_F770')


#Display our DM
SPC.dm1.display()


In [ ]:
#We can inject amplitude of 1 meter in 2 different actuators
SPC.dm1.set_actuator(11, 11, 1*u.m)
SPC.dm1.set_actuator(35, 35, 1*u.m)

SPC.dm1.display()

In [ ]:
#Display his influence on contrast
SPC.raw_contrast(display=True)

In [ ]:
#We can add all actuators we want inside grid (default 48x48) and replace value like this
SPC.dm1.set_actuator(11, 11, 0)
SPC.dm1.set_actuator(2, 24, 1*u.m)
SPC.dm1.set_actuator(36, 13, 1*u.cm)
SPC.dm1.display()

In [ ]:
#Flat DM
SPC.dm1.flatten()
SPC.dm1.display()

In [ ]:
# Finally you can use push_mode() method in PASTIS
cgi = RST() #RST telescop class in PASTIS

cgi.push_mode(750, 1)

cgi.sim.dm1.display()

## Pitch-scalling & Actuators grid
In this block, we use a method to verify if our DM works on pupil plan with the observation of amplifiate phase. We can also show pitch-scaling implementation.

In [ ]:
#Change nb! Number of actuators in line or/and column
########
nb = 48 #48 is the default number
#######

DM_grid = roman.CGI(mode='CHARSPC_F770', nbactuator=nb) #nbactuator is optionnal but nbactuator=48 by default

for i in range(nb):
    for j in range(nb):
        DM_grid.dm1.set_actuator(i, j, 1*u.nm)

#Get itermediates wawefront propagation 
_psf, inter = DM_grid.calc_psf(nlambda=1, return_intermediates=True)

grid = inter[3].phase #Get phase at the end of pupil plan

plt.imshow(grid**30) #Put a high power number to show grid

## Parametrization
We show how push actuator decreases our mean contrast inside the DH, but in top example, we use random values not optimal ones. But PASTIS need to be parameters to work in the linear regime, the 2 following block will help us to determine how we choose the right amplitude.

In [ ]:
points = 75 #Number of point in our curve
nb_actu = 6

abb_list = [0.2*i*10**(-5-2*i%5) for i in range(points)]
contrast = [i for i in range(points)]

sim_instance = roman.CGI(mode='CHARSPC_F770', nbactuator=nb_actu)
seg = (nb_actu**2)%2 + 1 #Choosen actuator to push

for i in range(points):
    sim_instance.dm1.flatten()
    actu_x, actu_y = util.seg_to_dm_xy(nb_actu, seg)
    sim_instance.dm1.set_actuator(actu_x, actu_y, abb_list[i])
    contrast[i] = sim_instance.raw_contrast()

# log results for a better reding
log_contrast = np.log10(contrast) 
log_abb = np.log10(abb_list)

In [ ]:
#Plotting our results
plt.scatter(log_abb,log_contrast)
plt.xlabel('log amplitude actuator(m)')
plt.ylabel('log contrast')
plt.axhline(-8.3566, color='grey', linewidth='2') # Contrast floor ()
plt.axvline(-6.113509275, color='grey', linewidth='2') # Wawelenght (770 nm)
plt.axvline(-8.301, color='grey', linewidth='2') # Chosen amplitude to inject (5 nm)
plt.show()

Contrast in function of the injected amplitude actuator learns us many things. We can distinguish 3 regimes:
- Contrast floor one with too low aberrations to deteriorate contrast
- Linear regime, C = K a
- Satured regime, start when aberrations are near of wawelenght
We can trust, we can choose a default amplitude to inject in middle of linear regime, but no. We must a value in knee of the curve because we will inject aberrations in multiple actuators. If we choose higher values PASTIS will be quickly out of range to make a good tolerancing. 